In [1]:
%load_ext cython
import numpy as np

Simple logic tests 

In [2]:
%%cython --annotate
cdef int a = 1
cdef int b = 1
cpdef foo():
    a == b

In [3]:
%timeit foo()

56 ns ± 0.413 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [4]:
c = 1
d = 1
%timeit c == d

68 ns ± 1.4 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


Functions to find numbers in array 

In [5]:
%%cython --annotate
import numpy as np
cimport numpy as np 
cpdef where_cy(np.ndarray[int, ndim=2] arr):
    cdef int i
    cdef int j
    cdef int n = 0
    cdef Py_ssize_t imax = arr.shape[0]
    cdef Py_ssize_t jmax = 2
    cdef list inds = []
    for i in range(imax):
        for j in range(jmax):
            if arr[i, j] == 0:
                inds.append([i, j])
                n+=1
        if n == 2:
            break
    return inds, i, j

In [6]:
def make_arr(n):
    A = np.arange(n)
    np.random.shuffle(A)
    roll = np.random.randint(n-1, size=1)[0]
    arr = np.c_[A, np.roll(A, roll)]
    return arr

In [7]:
def foo():
    arr = make_arr(251)
    where_cy(arr)
    
def foo2():
    arr = make_arr(251)
    np.where(arr == 0)

In [8]:
%timeit make_arr(251)
%timeit foo()
%timeit foo2()

83.5 µs ± 1.72 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
90.9 µs ± 1.03 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
97.8 µs ± 1.4 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Eularian path functions

In [9]:
from collections import defaultdict
def setEuPath(arr):
    g = defaultdict(set)
    order = np.zeros(len(arr), dtype=int)
    for v, w in arr:
        g[v].add(w)
        g[w].add(v)
    v = arr[0,0]
    i=0
    while True:
        try:
            w = g[v].pop()
        except KeyError:
            break
        g[w].remove(v)
        order[i] = v
        i+=1
        v = w
    return order

In [10]:
%%cython --annotate
import numpy as np
cimport numpy as np
def whereEuPath(np.ndarray[int, ndim=2] arr):
    cdef Py_ssize_t vmax = arr.shape[0]
    #cdef np.ndarray[int] order = np.zeros([vmax], dtype=int)
    cdef list order = []
    cdef np.ndarray[np.int64_t] x
    cdef np.ndarray[np.int64_t] y
    cdef int i, j, n
    cdef long long x1, x2, y1, y2
    i=0
    j=0
    for n in range(vmax):
        #order[n] = i
        order.append(i)
        ([x1, x2], [y1, y2]) = np.where(arr == arr[i, 1-j])
        if x1 == i:
            i = x2
            j = y2
        else:
            i = x1
            j = y1
    return order       
    

In [11]:
%%cython --annotate
import numpy as np
cimport numpy as np

cpdef testValue(int [:, :] arr, list rows, int val):
    cdef int i, x, y, newval
    i = 0
    for x in rows:
        for y in range(2):
            newval = arr[x,y]
            if newval == val:
                val = arr[x, 1-y]
                return (val, i)
        i+=1

cpdef np.ndarray[int] logEuPath(int [:, :] arr):
    cdef int vmax = arr.shape[0]
    rows = list(range(vmax))
    cdef int i, x
    cdef list order = []
    i = 0
    x = 0
    cdef int val = arr[i, 0]
    for n in range(vmax-1):
        del rows[i]
        order.append(val)
        (val, i) = testValue(arr, rows, val)
    order.append(val)
    return np.array(order)


In [12]:
%%cython --annotate
import numpy as np
cimport numpy as np

cpdef np.ndarray[int] logEuPath_v2(int [:, :] arr):
    cdef int vmax = arr.shape[0]
    cdef list rows = list(range(vmax))
    order_np = np.zeros([vmax], dtype=int)
    cdef int [:] order = order_np
    cdef int i = 0
    cdef int x, n, xmax
    cdef int val = arr[i, 0]
    cdef int nmax = vmax-1
    for n in range(nmax):
        del rows[i]
        order[n] = val
        i=0
        xmax = vmax - n + 1
        for x in rows: 
            if arr[x, 0] == val:
                val = arr[x, 1]
                break
            if arr[x, 1] == val:
                val = arr[x, 0]
                break
            i+=1
    order[n+1] = val
    return order_np

In [31]:
%%cython --annotate
import numpy as np
cimport numpy as np

cpdef np.ndarray[int] logEuPath_v3(np.ndarray[int, ndim=2] arr):
    cdef int vmax = arr.shape[0]
    cdef list rows = list(range(vmax))
    order_np = np.zeros([vmax], dtype=int)
    cdef int [:] order = order_np
    cdef int i = 0
    cdef int x, n, xmax
    cdef int val = arr[i, 0]
    cdef int nmax = vmax-1
    cdef int [:, :] arr_cy
    for n in range(nmax):
        del rows[i]
        order[n] = val
        i=0
        xmax = vmax - n + 1
        arr_cy = arr[rows, :]
        for e1, e2 in arr_cy: 
            if e1 == val:
                val = e2
                break
            if e2 == val:
                val = e1
                break
            i+=1
    order[n+1] = val
    return order_np

In [33]:
arr = make_arr(251)
%timeit setEuPath(arr)
#%timeit whereEuPath(arr)
%timeit logEuPath(arr)
%timeit logEuPath_v2(arr)
%timeit logEuPath_v3(arr)

586 µs ± 42 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
223 µs ± 21.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
159 µs ± 3.29 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
51.7 ms ± 5.03 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
e = np.loadtxt('edges.csv', delimiter=',').astype(float)
plane = 19.550487289428713

In [ ]:
def planeEdgeIntersect(edges, plane):
    axisInd = 2
    # Allocate intersect points array
    intersectPoints = np.zeros((edges.shape[0], 3))
    # Define the plane of intersect points
    intersectPoints[:, axisInd] = plane
    axesInd = np.array([0,1,2])[np.array([0,1,2]) != axisInd]
    for i in axesInd:
        intersectPoints[:, i] = (edges[:, i] +
                                 (plane - edges[:, axisInd]) *
                                 (edges[:, i+3] - edges[:, i]) /
                                 (edges[:, axisInd+3] - edges[:, axisInd]))
    return intersectPoints


In [ ]:
%%cython --annotate
cimport numpy as np
cimport cython
import numpy as np
cpdef np.ndarray[double, ndim=2] planeEdgeIntersect_cy(double [:, :] arr, double plane):
    cdef int axisInd = 2
    cdef int emax = arr.shape[0]
    intersectPoints_np = np.zeros((emax, 3))
    cdef double [:, :] intersectPoints = intersectPoints_np
    intersectPoints[:, axisInd] = plane
    cdef double e1, e2, e3, e4
    cdef Py_ssize_t i, j
    for i in range(emax):
        for j in range(2):
            e1 = arr[i, j]
            e2 = arr[i, axisInd]
            e3 = arr[i, j+3]
            e4 = arr[i, axisInd+3]
            intersectPoints[i, j] = e1 + (plane - e2) * (e3 - e1) / (e4 - e2)
    return intersectPoints_np

In [ ]:
%timeit planeEdgeIntersect(e, plane)
%timeit planeEdgeIntersect_cy(e, plane)